In [1]:
!apt-get -qq install poppler-utils tesseract-ocr
%pip install -q --user --upgrade pillow
%pip install -q unstructured["local-inference"]==0.6.6
%pip install -q "detectron2@git+https://github.com/facebookresearch/detectron2.git@e2ce8dc#egg=detectron2"
!pip install -q langchain chromadb sentence_transformers

Selecting previously unselected package poppler-utils.
(Reading database ... 122545 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Selecting previously unselected package tesseract-ocr-eng.
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2build2_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2build2) ...
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up tesseract-ocr (4.1.1-2b

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from collections import Counter
from unstructured.partition.pdf import partition_pdf
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}

In [5]:
elements = partition_pdf("/content/msc2020.pdf")
display(Counter(type(element) for element in elements))
print("")

Counter({unstructured.documents.elements.Text: 638,
         unstructured.documents.elements.ListItem: 303,
         unstructured.documents.elements.Title: 4817,
         unstructured.documents.elements.NarrativeText: 2104})

In [6]:
text_file = "\n\n".join([str(el) for el in elements])
print(text_file)

MSC2020



Mathematics Subject Classiﬁcation System

Associate Editors of Mathematical Reviews and zbMATH

00 General and overarching topics; collections

01 History and biography

03 Mathematical logic and foundations

05 Combinatorics

45 Integral equations

46 Functional analysis

47 Operator theory

49 Calculus of variations and optimal control; optimization

06 Order, lattices, ordered algebraic structures

51 Geometry

08 General algebraic systems

11 Number theory

12 Field theory and polynomials

13 Commutative algebra

14 Algebraic geometry

15 Linear and multilinear algebra; matrix theory

16 Associative rings and algebras

17 Nonassociative rings and algebras

18 Category theory; homological algebra

19 K



theory

20 Group theory and generalizations

22 Topological groups, Lie groups

26 Real functions

28 Measure and integration

30 Functions of a complex variable

31 Potential theory

32 Several complex variables and analytic spaces

33 Special functions

34 Ordinary diﬀ

In [7]:
codes = open("codes.txt", "w")
n = codes.write(text_file)
codes.close()

In [8]:
loader = TextLoader('/content/codes.txt')

In [9]:
hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [10]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [11]:
embeddings = hf
docsearch = Chroma.from_documents(texts, embeddings)

In [13]:
hf_llm = HuggingFaceHub(repo_id="EleutherAI/gpt-neo-2.7B", huggingfacehub_api_token="")

In [18]:
from langchain.prompts import PromptTemplate
prompt_template = """
You will be provided with the following {tag}, it is an MSC Classification Code,
You will also be provided a file that contains descriptions for each tag.
The heirarchy of the classification goes like this:
First is a 2 digit classification code e.g. 00-xx
Next a 3 digit classification code e.g. 00Axx and Finally a 5 digit classification code e.g. 00A05
In the absence of any of the tags do not place it in the description you return, just return what's available.

The text description for tag is:
"""
PROMPT = PromptTemplate(input_variables=["tag"], template=prompt_template)

In [21]:
qa = RetrievalQA.from_chain_type(llm=hf_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever())

In [23]:
tag = '00A07'
_PROMPT = PROMPT.format(tag=tag)
qa.run('hello how are you')

' I'